In [1]:
import pandas as pd
import numpy as np
import os
import sys
import polars as pl
import json
from joblib import Parallel, delayed
import deepchem
from tqdm import tqdm
import warnings
warnings.filterwarnings("ignore")

# Configure Polars 
cfg = pl.Config()
cfg.set_tbl_rows(20)
cfg.set_tbl_cols(50)
from sklearn.model_selection import StratifiedGroupKFold, GroupKFold, KFold, GroupShuffleSplit, ShuffleSplit, StratifiedKFold
import gc
import random
import catboost
from catboost.utils import create_cd

/tmp/ipykernel_41203/3612288966.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd
No normalization for SPS. Feature removed!
No normalization for AvgIpc. Feature removed!
Skipped loading some Tensorflow models, missing a dependency. No module named 'tensorflow'
wandb: WARNING W&B installed but not logged in.  Run `wandb login` or set the WANDB_API_KEY env variable.
Skipped loading modules with pytorch-geometric dependency, missing a dependency. No module named 'torch_geometric'
Skipped loading modules with pytorch-geometric dependency, missing a dependency. cannot import name 'DMPNN' from 'deepch

In [2]:
DELIMITER = '\t'
NUM_FEATURES = 2048
FEATURE_NAMES = [f'ECFP_{i}' for i in range(NUM_FEATURES)]

In [3]:
%%time
data = np.load('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ecfp6/train.npy')
data.shape

CPU times: user 3.03 ms, sys: 8 s, total: 8 s
Wall time: 8.01 s


(98415610, 256)

In [4]:
# np.unpackbits(data[:10], axis=1).shape

In [5]:
# data = np.unpackbits(data, axis=1)
# gc.collect()
# data.shape

In [6]:
(data!=0).mean()

0.3255673423926982

In [15]:
(0.325567 / 8) * (4 + 4 + 1)

0.366262875

In [ ]:
1

# Make a sample dataset to test

In [4]:
train_split_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/cv/v1/train.csv').collect()
train_split_df

label,index
i64,i64
0,363110
0,363111
0,363112
0,363113
0,363114
0,363115
0,363116
0,363118
0,363119


In [5]:
NUM_SAMPLES = 5_000_000
pos_idxs = train_split_df.filter(pl.col('label') > 0)['index'].to_numpy()
pos_idxs.shape

(1052622,)

In [6]:
remain_idxs = train_split_df.filter(pl.col('label') == 0)['index'].to_numpy()
num_neg = NUM_SAMPLES - len(pos_idxs)
print('Num neg:', num_neg)
np.random.seed(42)
neg_idxs = np.random.choice(remain_idxs, num_neg, replace = False)
sampled_idxs = pos_idxs.tolist() + neg_idxs.tolist()
len(sampled_idxs)

Num neg: 3947378


5000000

In [7]:
sampled_data = data[sampled_idxs]
sampled_data.shape

(5000000, 256)

In [28]:
val_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/cv/v1/val.csv',
                    dtypes = {'subset': pl.UInt8}).filter(pl.col('subset').is_in([0, 2, 3])).collect()
val_df

index,subset
i64,u8
486,0
489,0
494,0
498,0
504,0
519,0
527,0
528,0
534,0


In [29]:
val_idxs = val_df['index'].to_numpy()
val_idxs.shape

(299837,)

In [30]:
sampled_idxs = val_idxs

In [14]:
catboost.utils.create_cd(
      label=0,
      cat_features=list(range(3, NUM_FEATURES+3)),
      text_features=None,
      embedding_features=None,
      weight=None,
      baseline=None,
      doc_id=None,
      group_id=None,
      subgroup_id=None,
      timestamp=None,
      auxiliary_columns=None,
      feature_names={i+3: FEATURE_NAMES[i] for i in range(NUM_FEATURES)},
      output_path='/home/dangnh36/datasets/competitions/leash_belka/processed/features/catboost/train_ecfp6_2048_2048.cd'
)

In [15]:
catboost.utils.create_cd(
      label=0,
      text_features=None,
      embedding_features=None,
      weight=None,
      baseline=None,
      doc_id=None,
      group_id=None,
      subgroup_id=None,
      timestamp=None,
      auxiliary_columns=None,
      feature_names={i+3: FEATURE_NAMES[i] for i in range(NUM_FEATURES)},
      output_path='/home/dangnh36/datasets/competitions/leash_belka/processed/features/catboost/train_ecfp6_2048_2048_noctr.cd'
)

In [5]:
!cat /home/dangnh36/datasets/competitions/leash_belka/processed/features/train_ecfp6_2048_256_uint8.cd

0	Label	
3	Categ	ECFP_0
4	Categ	ECFP_1
5	Categ	ECFP_2
6	Categ	ECFP_3
7	Categ	ECFP_4
8	Categ	ECFP_5
9	Categ	ECFP_6
10	Categ	ECFP_7
11	Categ	ECFP_8
12	Categ	ECFP_9
13	Categ	ECFP_10
14	Categ	ECFP_11
15	Categ	ECFP_12
16	Categ	ECFP_13
17	Categ	ECFP_14
18	Categ	ECFP_15
19	Categ	ECFP_16
20	Categ	ECFP_17
21	Categ	ECFP_18
22	Categ	ECFP_19
23	Categ	ECFP_20
24	Categ	ECFP_21
25	Categ	ECFP_22
26	Categ	ECFP_23
27	Categ	ECFP_24
28	Categ	ECFP_25
29	Categ	ECFP_26
30	Categ	ECFP_27
31	Categ	ECFP_28
32	Categ	ECFP_29
33	Categ	ECFP_30
34	Categ	ECFP_31
35	Categ	ECFP_32
36	Categ	ECFP_33
37	Categ	ECFP_34
38	Categ	ECFP_35
39	Categ	ECFP_36
40	Categ	ECFP_37
41	Categ	ECFP_38
42	Categ	ECFP_39
43	Categ	ECFP_40
44	Categ	ECFP_41
45	Categ	ECFP_42
46	Categ	ECFP_43
47	Categ	ECFP_44
48	Categ	ECFP_45
49	Categ	ECFP_46
50	Categ	ECFP_47
51	Categ	ECFP_48
52	Categ	ECFP_49
53	Categ	ECFP_50
54	Categ	ECFP_51
55	Categ	ECFP_52
56	Categ	ECFP_53
57	Categ	ECFP_54
58	Categ	ECFP_55
59	Categ	ECFP_56
60	Categ	ECFP_57
61	Categ	ECFP_58
62	Ca

In [9]:
# NUM_SAMPLES = 98415610
# NUM_SAMPLES = 5_000_000
# NUM_SAMPLES = 299_837
NUM_SAMPLES = 66710159
NUM_FEATURES = 256
DTYPE = np.uint8

label = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/train_v2.csv',
                   dtypes = {
                    'BRD4': pl.UInt8,
                    'HSA': pl.UInt8,
                    'sEH': pl.UInt8,
                }).select(pl.col('BRD4', 'HSA', 'sEH')).collect().to_numpy()[sampled_idxs]
label.shape

(66710159, 3)

In [10]:
all_data = np.empty((NUM_SAMPLES, NUM_FEATURES + 3), dtype = np.uint8)
all_data.shape

(66710159, 259)

In [11]:
%%time
all_data[:, :3] = label

CPU times: user 807 ms, sys: 17.4 s, total: 18.2 s
Wall time: 18.2 s


In [13]:
all_data[:, 3:] = data[sampled_idxs]

In [14]:
all_data.shape

(66710159, 259)

In [34]:
%%time
all_data[:, 3:] = np.unpackbits(data[sampled_idxs], axis = 1)
# all_data[:, 3:] = data[sampled_idxs]

CPU times: user 226 ms, sys: 144 ms, total: 370 ms
Wall time: 366 ms


In [35]:
all_data.shape

(299837, 2051)

In [23]:
np.save('/home/dangnh36/datasets/competitions/leash_belka/processed/features/_train_ecfp6_2048_256_uint8_5M.npy', all_data)

In [36]:
# df = pl.from_numpy(data, schema=[(feature_name, pl.UInt8) for feature_name in feature_names])
# df

# header = DELIMITER.join(features)
header = ''
np.savetxt('/home/dangnh36/datasets/competitions/leash_belka/processed/features/catboost/_val_ecfp6_2048_2048.tsv',
           all_data,
           fmt = '%d',
           delimiter='\t',
           header = header)

In [12]:
# header = ''
# np.savetxt('/home/dangnh36/datasets/competitions/leash_belka/processed/features/catboost/train_ecfp6_2048_256_uint8_500k.tsv',
#            all_data[:500000],
#            fmt = '%d',
#            delimiter='\t',
#            header = header)

# Test

In [7]:
catboost.utils.create_cd(
      label=None,
      cat_features=list(range(0, NUM_FEATURES)),
      text_features=None,
      embedding_features=None,
      weight=None,
      baseline=None,
      doc_id=None,
      group_id=None,
      subgroup_id=None,
      timestamp=None,
      auxiliary_columns=None,
      feature_names={i+0: FEATURE_NAMES[i] for i in range(NUM_FEATURES)},
      output_path='/home/dangnh36/datasets/competitions/leash_belka/processed/datasets/catboost/ecfp6/test_ctr.cd'
)

In [4]:
%%time
data = np.load('/home/dangnh36/datasets/competitions/leash_belka/processed/features/ecfp6/test.npy')
data.shape

CPU times: user 0 ns, sys: 177 ms, total: 177 ms
Wall time: 1.28 s


(878022, 256)

In [5]:
data = np.unpackbits(data, axis = 1)
data.shape

(878022, 2048)

In [6]:
header = ''
np.savetxt('/home/dangnh36/datasets/competitions/leash_belka/processed/datasets/catboost/ecfp6/test.tsv',
           data,
           fmt = '%d',
           delimiter='\t',
           header = header)

In [11]:
data.shape

(98415610, 256)

In [4]:
cv_train_df = pl.scan_csv('/home/dangnh36/datasets/competitions/leash_belka/processed/cv/v1/train.csv').with_row_index('_index').collect()
print(cv_train_df.shape)
cv_train_df.head(5)

(66710159, 3)


_index,index,label
u32,i64,i64
0,363110,0
1,363118,0
2,363119,0
3,363122,0
4,363123,0


In [5]:
sampled_idxs = cv_train_df['index'].to_numpy()

In [6]:
sampled_idxs.shape

(66710159,)

In [13]:
data = data[cv_train_df['index'].to_numpy()]
data.shape

(66710159, 256)

In [15]:
all_data.shape

(66710159, 259)

In [16]:
header = ''
np.savetxt('/home/dangnh36/datasets/competitions/leash_belka/processed/datasets/catboost/ecfp6_packbits/_train_66M.tsv',
           all_data,
           fmt = '%d',
           delimiter='\t',
           header = header)